In [142]:
import pandas as pd
from shapely.geometry import Polygon
from shapely.wkt import loads
import xlwings as xw

from src.validation import SearchAmbiguity

In [108]:
selected_cells = xw.apps.active.selection
selected_cells.value

7.0

In [17]:
selected_cells = xw.apps.active.selection
wkt = selected_cells.value
polygon: Polygon = loads(wkt)

In [166]:
sheet_result = 'Зона 1'

In [152]:
sheet_autocad = xw.sheets['Автокад']
autocad_df = sheet_autocad.range('A1').expand().options(pd.DataFrame, header=1, index=False).value
autocad_df['number_position'] = autocad_df['number_position'].apply(lambda x: loads(x))
autocad_df['geometry'] = autocad_df['geometry'].apply(lambda x: loads(x))

autocad_df

,index,Номер точки,number_position,split_number,type,geometry,size
0,0,3,POINT (3108.838205298518 1707.285109871409),3,Point,POINT (3108.838205298518 1707.285109871409),NaN
1,1,4,POINT (3114.150988844972 1707.4695572873902),4,Point,POINT (3114.150988844972 1707.4695572873902),NaN
2,2,5,POINT (3115.690642463274 1707.5867820573599),5,Point,POINT (3115.690642463274 1707.5867820573599),NaN
3,3,6,POINT (3115.1004453425708 1706.18062074892),6,Point,POINT (3115.1004453425708 1706.18062074892),NaN
4,4,7,POINT (3114.699609216242 1705.0905155704238),7,Point,POINT (3114.699609216242 1705.0905155704238),NaN
5,5,10,POINT (3104.043722283615 1698.543651078497),10,Point,POINT (3104.043722283615 1698.543651078497),NaN
6,6,11,POINT (3105.090899215862 1695.9481208535972),11,Point,POINT (3105.090899215862 1695.9481208535972),NaN
7,7,12,POINT (3106.332887793542 1693.7013226121771),12,Point,POINT (3106.332887793542 1693.7013226121771),NaN
8,8,14,POINT (3104.361115387819 1692.888423763281),14,Point,POINT (3104.361115387819 1692.888423763281),NaN
9,9,13,POINT (3107.4888707400646 1689.420932022136),13,Point,POINT (3107.4888707400646 1689.420932022136),NaN


In [147]:
sheet_zones = xw.sheets['Зоны']
zones_df = sheet_zones.range('A1').expand().options(pd.DataFrame, header=1, index=False).value

zones_df

,index,name,geometry
0,0.0,Зона 1,"POLYGON ((3089.86853687141 1709.814916853105, ..."
1,1.0,Зона 2,MULTIPOLYGON (((3117.421038375149 1688.1591344...
2,2.0,Зона 3,POLYGON ((3122.978988696761 1704.0782505312322...


In [171]:
zone_names = zones_df.name.tolist()
zone_names.remove(sheet_result)

zone_names

['Зона 2', 'Зона 3']

In [172]:
from win32com.universal import com_error

used_split_numbers = []
for zone_name in zone_names:
    try:
        sheet_zone = xw.sheets[zone_name]
    except com_error:
        continue
    split_numbers_series = sheet_zone.range('A1').expand().options(pd.DataFrame, header=1, index=False).value['split_number']
    used_split_numbers.extend(split_numbers_series.tolist())

used_split_numbers

['21',
 '22',
 '23',
 '24',
 '25',
 '26',
 '26а',
 '27.1',
 '27.2',
 '27.3',
 '8',
 '9',
 '1',
 '2']

In [173]:
autocad_df_not_used = autocad_df[~autocad_df['split_number'].isin(used_split_numbers)]

autocad_df_not_used

,index,Номер точки,number_position,split_number,type,geometry,size
0,0,3,POINT (3108.838205298518 1707.285109871409),3,Point,POINT (3108.838205298518 1707.285109871409),NaN
1,1,4,POINT (3114.150988844972 1707.4695572873902),4,Point,POINT (3114.150988844972 1707.4695572873902),NaN
2,2,5,POINT (3115.690642463274 1707.5867820573599),5,Point,POINT (3115.690642463274 1707.5867820573599),NaN
3,3,6,POINT (3115.1004453425708 1706.18062074892),6,Point,POINT (3115.1004453425708 1706.18062074892),NaN
4,4,7,POINT (3114.699609216242 1705.0905155704238),7,Point,POINT (3114.699609216242 1705.0905155704238),NaN
5,5,10,POINT (3104.043722283615 1698.543651078497),10,Point,POINT (3104.043722283615 1698.543651078497),NaN
6,6,11,POINT (3105.090899215862 1695.9481208535972),11,Point,POINT (3105.090899215862 1695.9481208535972),NaN
7,7,12,POINT (3106.332887793542 1693.7013226121771),12,Point,POINT (3106.332887793542 1693.7013226121771),NaN
8,8,14,POINT (3104.361115387819 1692.888423763281),14,Point,POINT (3104.361115387819 1692.888423763281),NaN
9,9,13,POINT (3107.4888707400646 1689.420932022136),13,Point,POINT (3107.4888707400646 1689.420932022136),NaN
